 Login to your azure account az login - device

 Ensure you change the Kernel to Python 3.10 PyTorch and Tensorflow

In [ ]:
! pip install onnxruntime-genai==0.7.0 --pre

In [ ]:
import onnxruntime_genai as og
import numpy as np
import os

In [ ]:
model_folder = "./models/phi-4-mini/onnx/model"

In [ ]:
model = og.Model(model_folder)

In [ ]:
adapters = og.Adapters(model)
adapters.load('./models/phi-4-mini/onnx/model/adapter_weights.onnx_adapter', "qa_choice")

In [ ]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

In [ ]:
search_options = {}
search_options['max_length'] = 102
search_options['past_present_share_buffer'] = False

In [ ]:
input = "Answer the following multiple-choice question by selecting the correct option.\n\nQuestion: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?\nAnswer Choices:\n(A) ignore\n(B) enforce\n(C) authoritarian\n(D) yell at\n(E) avoid"

In [ ]:
chat_template = "</system>You are a helpful assistant. Your output should only be one of the five choices: 'A', 'B', 'C', 'D', or 'E'.<|end|><|user|>{input}<|end|><|assistant|>"

In [ ]:
prompt = f'{chat_template.format(input=input)}'

In [ ]:
input_tokens = tokenizer.encode(prompt)

In [ ]:
params = og.GeneratorParams(model)
params.set_search_options(**search_options)
generator = og.Generator(model, params)

In [ ]:
generator.set_active_adapter(adapters, "qa_choice")

In [ ]:
generator.append_tokens(input_tokens)

In [ ]:
while not generator.is_done():
            generator.generate_next_token()

            new_token = generator.get_next_tokens()[0]
            print(tokenizer_stream.decode(new_token), end='', flush=True)